# Проект спринта №7. Анализ для игры "Секреты темнолесья".

### Цели и задачи проекта

В этот тетрадке используем датасет `/datasets/new_games.csv`, который содержит информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр. На данных датасета мы проведем анализ развития игровой индустрии с 2000 по 2013 год: категоризируем игры на оценках пользователей и выделим топ-7 платформ по количеству игр, выпущенных за этот период.

### Описание данных

В проекте будут использованы данные датасета `/datasets/new_games.csv` с таким описанием:

- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка и знакомство с данными
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных

## Загрузка и знакомство с данными

Загрузим необходимые библиотеки для анализа данных и данные из датасета `/datasets/new_games.csv`. Затем выведем основную информацию о данных с помощью метода info() и первые строки датафрейма.

In [1]:
# Импортируем библиотеку pandas
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

In [2]:
# Выгружаем данные из датасета datasets/new_games.csv в датафрейм games
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
# Выводим информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые строки датафрейма на экран
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


Датасет `/datasets/new_games.csv` содержит 11 столбцов и 16956 строк, в которых преставлена информация о видеоиграх. Данные полностью соотвествуют описанию и отображают содержимое. Названия столбцов написаны с разными регистрами и пробелами, что может вызвать неудобства при обработке данных.

Изучим типы данных и их корректность:
- **Object**. Семь столбцов представлены типом `object`:
  - `Name`, `Platform`, `Genre` и `Rating` содержат строковую информацию (название игры, название платформы, жанр игры, рейтинг организации ESRB), что логично для текстовых данных. Тип `object` выбран правильно.
  - `EU sales`, `JP sales` и `User Score` содержат числовую информацию (продажи в Европе, продажи в Японии, оценка пользователей). Для них рекомендуется использовать тип данных `flot64`. Это позволит корректно проводить над ними математические операции в дальнейшем анализе.
- **Float64**. Четыре столбца имеют тип данных `float64`:
  - `Year of Release` хранит информацию о годе выпуска игры. Тип данных можно сменить на `int16`, который облегчит фильтрацию по годам.
  - `NA sales`, `Other sales` и `Critic Score` хранят числовую информацию (продажи в Северной Америке, продажи в других странах, оценка критиков). Выбранный тип данных `float64` корректен для данных столбцов.

После анализа типов данных видно, что большинство столбцов представлены корректными типами данных. Однако для оптимизации нужно преобразовать некоторые столбцы к вещественному типу и целочисленному. Помимо этого необходимо привести названия столбцов к стилю snake case для корректного написания, так как сейчас нет единого формата для них: используются пробелы и разные регистры.

Данные содержат пропуски в шести столбцах: `Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score` и `Rating`. Следует оценить долю потерянной информации и учесть при дальнейшем анализе. 

## Проверка ошибок в данных и их предобработка

### Работа с названиями столбцов датафрейма

Для начала проверим стиль написания столбцов датафрейма и преобразуем его при необходимости.

In [5]:
# Выведем на экран названия всех столбцов датафрейма и проверим их стиль написания
print(games.columns)

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')


Для удобства работы с датафреймом приведем все названия столбцов к единому стилю snake case. 

In [6]:
# Приведем названия столбцов к единому виду
games.columns = games.columns.str.lower().str.replace(' ','_') 

In [7]:
# Проверим наши преобразования
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Теперь названия столбцов приведены к единому стилю, что облегчит анализ информации в последующем.

### Работа с типами данных

Столбцы `eu_sales`, `jp_sales` и `user_score` содержат числовую информацию (продажи в Европе, продажи в Японии, оценка пользователей) и корректный тип данных для них `float64`. Однако сейчас они находятся в типе `object`. Возможно, они содержат строковые значения и их следует заменить на Nan для преобразования к `float64`.

In [8]:
# Выводим уникальные значения в столбце eu_sales
unique_values = games['eu_sales'].unique()
display('Уникальные значения в eu_sales:')
display(unique_values)

'Уникальные значения в eu_sales:'

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

В столбце `eu_sales` помимо чисел находится значение 'unknown'. Заменим его на пустое значение - None.

In [9]:
# Заменяем на None
games['eu_sales'] = games['eu_sales'].replace('unknown', None)

In [10]:
# Приведем столбец к численному типу данных
games['eu_sales'] = pd.to_numeric(games['eu_sales'])

In [11]:
# Выводим уникальные значения в столбце jp_sales
unique_values = games['jp_sales'].unique()
display('Уникальные значения в jp_sales:')
display(unique_values)

'Уникальные значения в jp_sales:'

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [12]:
# Также значения 'unknown' заменяем на None
games['jp_sales'] = games['jp_sales'].replace('unknown', None)

In [13]:
# Приведем столбец к численному типу данных
games['jp_sales'] = pd.to_numeric(games['jp_sales'])

In [14]:
# Выводим уникальные значения в столбце user_score
unique_values = games['user_score'].unique()
display('Уникальные значения в user_score:')
display(unique_values)

'Уникальные значения в user_score:'

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In [15]:
# Аналогично 'unknown' и 'tbd' заменяем на None
games['user_score'] = games['user_score'].replace('unknown', None)
games['user_score'] = games['user_score'].replace('tbd', None)

In [16]:
# Приведем столбец к численному типу данных
games['user_score'] = pd.to_numeric(games['user_score'])

Проверим наши преобразования, вызвав метод info().

In [17]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


Таким образом, мы заменили аномальные строковые значения в столбцах `eu_sales`, `jp_sales` и `user_score` на пустые значения, чтобы обработать эти пропуски в дальнейшем, и привели столбцы к численным типам данных для выполнения корректных математических операций.

### Работа с пропусками в данных

In [18]:
# Выводим количество пропущенных строк датафрейма
games.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [19]:
# Подсчитываем процент строк с пропусками
games.isna().sum() / games.shape[0] * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Датафрейм содержит восемь столбцов с пропусками:
- `name` и `genre` содержат всего по два пропуска, что составляет менее 1% данных. Скорее всего, это вызвано технической ошибкой, и в связи с отсутствием информации о названии игры становится невозможным определить ее жанр. Эти строки можно удалить, они не критично повлияют на анализ.
- Столбец `year_of_release` хранит 275 пропусков (1,62%). Пропуски так же случайны, их доля невелика, поэтому удалим эти строки.
- В столбцах `eu_sales` и `jp_sales` содержится 6 и 4 пропуска (0.04% и 0.02%) соотвественно. Их можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.
- Пропуски в столбцах `critic_score` и `user_score` в абсолютном значении составляют 8714 и 9268, а в относительном 51,59% и 54,66% от всех данных соотвественно. Довольно значительная доля информации пропала. Причиной этому может стать отсутствие оценки критиков и пользователей этих игр, то есть пропуски неслучайны. Значения можно заменить индикатором (-1).
- Столбец `rating` содержит 6871 пропуск (40,52%). Аналогично предыдущим столбцам пропуски могут быть вызваны неслучайным отсутствием информации. Для присвоения рейтинга ESRB издатель игры должен прислать этой организации материал для оценки. Возможно, создатели этих игр не совершали этого. Заменим пропуски строкой 'нет данных'. 

In [20]:
# Сохраним исходное число строк датафрейма 
games_lines = games.shape[0]

In [21]:
# Удалим из датафрейма строки с пропусками в столбцах `name`, `genre` и `year_of_release`
games = games.dropna(subset = ['name', 'genre', 'year_of_release'])

После того, как мы удалили пропуски в столбце `year_of_release`, можно привести его к типу `int16`.

In [22]:
games['year_of_release'] = games['year_of_release'].astype('int16')

Чтобы заменить пропуски в столбцах `eu_sales` и `jp_sales` на среднее значение, исходя из названия платформы и года выпуска, создадим переменную `mean_eu_sales` и `mean_jp_sales`, используя группировку и метод `transform`. Заполним пропуски в столбцах этими переменными с помощью функции `fillna()`.

In [23]:
# Заменим пропуски в столбцe `eu_sales`
mean_eu_sales = games.groupby(['platform','year_of_release'])['eu_sales'].transform('mean')
games['eu_sales'] = games['eu_sales'].fillna(mean_eu_sales)

In [24]:
# Заменим пропуски в столбцe `jp_sales`
mean_jp_sales = games.groupby(['platform','year_of_release'])['jp_sales'].transform('mean')
games['jp_sales'] = games['jp_sales'].fillna(mean_jp_sales)

Теперь заменим пропуски в столбцах `critic_score` и `user_score` значениями-индикаторами. Используем -1, так как оно не может быть использовано в данных.

In [25]:
games['critic_score'] = games['critic_score'].fillna(-1)
games['user_score'] = games['user_score'].fillna(-1)

Аналогично со столбцом `rating` - заменяем пропуски на строку 'нет данных'.

In [26]:
games['rating'] = games['rating'].fillna('нет данных')

Проверим информацию о датафрейме после обработки пропусков с помощью метода info().

In [27]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  int16  
 3   genre            16679 non-null  object 
 4   na_sales         16679 non-null  float64
 5   eu_sales         16679 non-null  float64
 6   jp_sales         16679 non-null  float64
 7   other_sales      16679 non-null  float64
 8   critic_score     16679 non-null  float64
 9   user_score       16679 non-null  float64
 10  rating           16679 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.4+ MB


Все пропуски в датафрейме заполнены. 

Займемся обработкой неявных дубликатов. Для начала изучим уникальные значения категориальных данных - столбцы `name`, `platform`, `year_of_release`, `genre` и `rating`, чтобы вычвить опечатки или данные с разным способом написания.

In [28]:
# Уникальные значения `platform`
unique_platforms = games['platform'].unique()
display(unique_platforms)

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

Дубликатов в столбце `platform` не выявлено. Переходим к проверке `year_of_release`.

In [29]:
# Уникальные значения `year_of_release`
unique_years = games['year_of_release'].unique()
display(unique_years)

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000, 1995, 1991,
       1981, 1987, 1980, 1983], dtype=int16)

Так же нет дубликатов.

In [30]:
# Уникальные значения `genre`
unique_genres = games['genre'].unique()
display(unique_genres)

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

Заметим, что многие значения столбца `genre` - одинаковые жанры, только написаные в разном регистре. Проведем нормализацию данных, приведя строки к нижнему регистру.

In [31]:
games['genre'] = games['genre'].str.lower()

In [32]:
# Уникальные значения `rating`
unique_ratings = games['rating'].unique()
display(unique_ratings)

array(['E', 'нет данных', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

Неявных дубликатов в столбце `rating` нет.

Приведем столбец `name` к единому стилю, чтобы при проверке на явные дубликаты, можно было найти одинаковые строки.

In [33]:
games['name'] = games['name'].str.lower()

Проверим, есть ли в датафрейме явные дубликаты, используя метод duplicated().

In [34]:
# Считаем количество дублированных строк 
games.duplicated().sum()

235

В датафрейме 235 явных дубликатов. Удалим их с помощью метода drop_dublicates().

In [35]:
#Удаляем дубликаты
games.drop_duplicates(inplace = True)

Проверим есть ли неявные дубликаты данных по столбцам `name`, `platform`, `year_of_release`. Комбинации значений в них должны быть уникальными для корректной обработки данных.

In [36]:
duplicated_names = games.duplicated(subset = ['name', 'platform', 'year_of_release']).sum()
display(f"Найдено {duplicated_names} дубликатов")

'Найдено 1 дубликатов'

In [37]:
# Удаляем эти дубликаты
games.drop_duplicates(subset = ['name', 'platform', 'year_of_release'], inplace = True)

Посчитаем количество удаленных строк после всей предобработки данных. Для этого вернемся к переменной `games_lines`, сохраненной нами в начале работы. Вычтем из нее количество строк обработанного датафрейма и найдем абсолютное и относительное значение удаленных строк.

In [38]:
# Количество удаленных строк
deleted_lines = games_lines - games.shape[0]
display(f"Количество удаленных строк: {deleted_lines}")

# Процент удаленных строк
percent_of_deleted_lines = deleted_lines / games_lines * 100
display(f"Процент удаленных строк: {percent_of_deleted_lines}")

'Количество удаленных строк: 513'

'Процент удаленных строк: 3.0254777070063694'

В результате обработки данных были выполнены следующие действия:
- Названия столбцов датафрейма были приведены к единому стиля snake case
- Столбцы `eu_sales`, `jp_sales` и `user_score` были приведены к типу данных `float64`, столбец `year_of_release` - к типу данных `int16`
- Все пропуски были обработаны: строки столбцов `name`, `genre` и `year_of_release`, содержащих пропуски, были удалены, так как составляли небольшую часть данных. Пустые значения в `eu_sales` и `jp_sales` были заменены на средние в зависимости от платформы и года выпуска игры. Пропуски в `critic_score` и `user_score` заполнили значением=индикатором (-1), а в столбце `rating` строкой 'нет данных'.
- В столбце `genres` мы нашли неявные дубликаты, появившиеся в результате использования разного регистра. Мы провели нормализацию данных, чтобы избежать ошибок в дальнейшем анализе.
- Мы нашли 235 явных дубликатов и 1 неявный. Провели очистку датафрейма от них.
- Вся обработка данных потребовала от нас удаления 513 строк, что составило 3,03% от исходного датафрейма.

## Фильтрация данных

Отберем данные за период с 2000 по 2013 года включительно и сохраним новый срез данных в отдельном датафрейме `df_actual`.

In [39]:
# Фильтруем датафрейм и сохраняем в новый
df_actual = games.loc[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)]

In [40]:
# Выведем информацию о новом датафрейме
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12780 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12780 non-null  object 
 1   platform         12780 non-null  object 
 2   year_of_release  12780 non-null  int16  
 3   genre            12780 non-null  object 
 4   na_sales         12780 non-null  float64
 5   eu_sales         12780 non-null  float64
 6   jp_sales         12780 non-null  float64
 7   other_sales      12780 non-null  float64
 8   critic_score     12780 non-null  float64
 9   user_score       12780 non-null  float64
 10  rating           12780 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.1+ MB


После фильтрации мы получили новый датафрейм `df_actual`. В нем осталось 11 столбцов и 12780 строки, в которых преставлена обработанная информация о видеоиграх, выпущенных в период с 2000 по 2013 года включительно. Так как этот датафрейм создан на основе предыдущего, то описание столбцов, стиль написания и содеражащаяся в нем информация корректны.

Изучим типы данных:
- **Object**. Четыре столбца представлены типом `object`:
  - `name`, `platform`, `genre` и `rating` содержат строковую информацию (название игры, название платформы, жанр игры, рейтинг организации ESRB), что логично для текстовых данных. Тип `object` выбран правильно.
- **Float64**. Шесть столбцов имеют тип данных `float64`:
  - `na_sales`, `eu_sales`, `jp_sales`, `other_sales`, `critic_score` и `user_score` хранят числовую информацию (продажи в Северной Америке, продажи в Европе, продажи в Японии, продажи в других странах, оценка критиков, оценка пользователей). Выбранный тип данных `float64` корректен для данных столбцов.
- **Int16**. Один столбец хранит информацию типа `int16`:
  - `year_of_release` содержит года выпуска игр, которые представлены целыми числами.
  
Данные не содежат пропусков и дубликатов, так как получены из обработанного датафрейма. Можно приступать к анализу.

## Категоризация данных

Проведем категоризацию данных. Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала). 

In [41]:
df_actual['user_category'] = pd.cut(df_actual['user_score'], bins = [0, 3, 8, 10.01], labels = ['низкая', 'средняя', 'высокая'], right = False)

Разделим все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [42]:
df_actual['critics_category'] = pd.cut(df_actual['critic_score'], bins = [0, 30, 80, 100.01], labels = ['низкая', 'средняя', 'высокая'], right = False)

Выведем первые пять строк полученного датафрейма, чтобы посмотреть результат преобразований.

In [43]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_category,critics_category
0,wii sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая,средняя
2,mario kart wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая,высокая
3,wii sports resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая,высокая
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая,высокая
7,wii play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя,средняя


Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [44]:
# Группируем данные по `platform`, применяем агрегирующую функцию count()
top_7_platforms = df_actual.groupby('platform')['name'].count()

In [45]:
# Cортируем названия платформ по убыванию количества выпущенных игр
top_7_platforms = top_7_platforms.sort_values(ascending = False)

In [46]:
# Выбираем топ-7 платформ по количеству выпущенных игр, используя срез данных
top_7_platforms = top_7_platforms[:7]

In [47]:
# Выводим топ-7 платформ на экран
display(top_7_platforms)

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1086
GBA      811
Name: name, dtype: int64

В первом столбце топа предствлены названия платформ, а во втором - количество игр, выпущенных в период с 2000 по 2013 год включительно. В топ вошли PS2 (2127 запусков), DS(2120), Wii (1275), PSP (1180), X360 (1121), PS3 (1086) и GBA (811).  


## Основные выводы и результаты

Были загружены данные `/datasets/new_games.csv`. Они содержат 11 столбцов и 16956 строк, в которых представлена информация о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр. Для удобства работы с данными названия столбцов были приведены к единому стилю snake case. При первичном знакомстве с данными и их предобработкой получили такие результаты:
- После замены строковых значениях в числовых столбцах было получено: восемь столбцов (`name`, `genre`, `year_of_release`, `eu_sales`, `jp_sales`, `critic_score`, `user_score`, `rating`) содержат пропуски. Максимальное значение пропущенных данных в столбце `user_score` - 54.66%. 
- Для оптимизации работы с данными в датафрейме были произведены следующие изменения типов данных:
  - `year_or_release`: тип изменен с `float64` на `int16`
  - `eu_sales`, `jp_sales` и `user_score`: тип изменен с `object` на `float64`
- В ходе обработки данных было удалено 513 строк, что составило 3,03% от исходного датафрейма.
- После очистки данных исходного датафрема был создан новый `df_actual`, в который вошел период выпуска игр с 2000 по 2013 года включительно. В нем осталось 11 столбцов и 12780 строки. 
- Для дополнительной работы с данными в датафрейме `df_actual` были созданы дополнительные столбцы:
  - `user_category`: столбец с категорией игр по оценкам пользователей (столбец `user_score`) - высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).
  - `critics_category`: столбец с категорией игр по оценкам критиков (`critic_score`) - высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).
- Был составлен топ-7 платформ по количеству запусков игр за отведенный период. В него вошли платформы PS2 (2127 запусков), DS(2120), Wii (1275), PSP (1180), X360 (1121), PS3 (1086) и GBA (811)
